# ML2 Semestral Project - Football O/U 2.5
**Authors:** Phuong Nhi Tranová, Vít Maruniak, Šimon Slánský, Radim Škoukal, Ondřej Zetek, Martin Kareš, Jan Korčák, Jakub Maličkay, Jáchym Janouch  
**Course:** FIS 4IT344 Machine Learning 2 (2025/2026)  
**Goal:** Compare baseline (current features) vs extended (richer features) models for O/U 2.5 goals across markets; translate accuracy gains into optimal profit and **maximum data subscription price per country** *.  



---


***maximum data subscription price per country**
- the most money our company should be willing to pay for that country's additional data
- that's how much extra profit the improved model generates
- baseline model → accuracy = A₀
    - Generates profit Π*(A₀)
- extended model → accuracy = A₁
    - Generates profit Π*(A₁)
- profit improvement = ΔΠ = Π(A₁) − Π(A₀)*
    - basically how much more money the comany earns each year by using the better data
- the maximum data subscription price per country = ΔΠ


# 0. Imports and paths

### 0.1 Imports

In [ ]:
import os, glob, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import pickle

### 0.2 Library parameters

In [ ]:
plt.rcParams["figure.figsize"] = (8,5)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### 0.3 Paths

In [ ]:
DATA_DIR = "./data"  
OUTPUT_DIR = f"./processed"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Data load

In [ ]:
def load_all_matches(data_dir: str) -> pd.DataFrame:
    csv_files = glob.glob(os.path.join(data_dir, "**", "*.csv"), recursive=True)
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found under {data_dir}")

    frames = []
    for fp in csv_files:
        # extract path info
        rel = os.path.relpath(fp, data_dir)
        parts = Path(rel).parts
        country = parts[0] if len(parts) >= 1 else None
        league  = parts[1] if len(parts) >= 2 else None
        season_file = parts[2] if len(parts) >= 3 else None
        season = os.path.splitext(season_file)[0] if season_file else None

        # read and rename
        try:
            df = pd.read_csv(fp, low_memory=False)
        except Exception as e:
            print(f"Skipping {fp}: {e}")
            continue

        frames.append(df)

    all_df = pd.concat(frames, ignore_index=True, sort=False)
    return all_df

# run the loader
all_matches = pd.DataFrame(load_all_matches(DATA_DIR))
print(all_matches.columns.tolist())
print(all_matches.shape)
display(all_matches.head())

# 2. Exploratory Data Analysis

Before proceeding with data cleaning, let's understand our data better through comprehensive exploratory data analysis. This will help us make informed decisions about preprocessing steps.

### 2.1 Data Shape and Overview

In [ ]:
print(f"Dataset shape: {all_matches.shape}")
print(f"Number of seasons/countries covered:")
print(f"Countries: {all_matches['Div'].str[:-1].nunique()}")
print(f"Leagues: {all_matches['Div'].nunique()}")
print(f"Date range: {all_matches['Date'].min()} to {all_matches['Date'].max()}")

# Check basic statistics
print(f"\nBasic goal statistics:")
print(f"Total goals per match stats:")
total_goals = all_matches['FTHG'] + all_matches['FTAG']
print(total_goals.describe())

print(f"\nOver/Under 2.5 goals distribution:")
over_2_5 = (total_goals > 2.5).astype(int)
print(f"Over 2.5: {over_2_5.sum()} ({over_2_5.mean():.2%})")
print(f"Under 2.5: {(~over_2_5.astype(bool)).sum()} ({(1-over_2_5.mean()):.2%})")

Great! Our target variable (Over/Under 2.5 goals) is perfectly balanced with almost exactly 50/50 split, which is ideal for classification.

### 2.2 Missing Values Analysis

In [ ]:
# Detailed missing values analysis
missing_analysis = pd.DataFrame({
    'column': all_matches.columns,
    'missing_count': all_matches.isnull().sum(),
    'missing_percentage': (all_matches.isnull().sum() / len(all_matches)) * 100,
    'dtype': all_matches.dtypes
})

# Filter to show only columns with missing values
missing_analysis = missing_analysis[missing_analysis['missing_count'] > 0].sort_values('missing_percentage', ascending=False)

print(f"Columns with missing values: {len(missing_analysis)}")
print(f"Total columns: {len(all_matches.columns)}")
print(f"\nTop 20 columns with highest missing percentage:")
display(missing_analysis.head(20))

# Check missing patterns in key variables
key_stats = ['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
print(f"\nMissing data in key match statistics:")
for stat in key_stats:
    if stat in all_matches.columns:
        missing_pct = (all_matches[stat].isnull().sum() / len(all_matches)) * 100
        print(f"{stat}: {missing_pct:.1f}%")

The missing data analysis reveals that:
1. **Betting odds** have the highest missing percentages (80%+) - this is expected as not all bookmakers operate in all leagues/seasons
2. **Key match statistics** (shots, corners, fouls, cards) have very low missing rates (<0.1%), which is excellent for our modeling
3. Most missing data is in betting-related columns, which we can handle appropriately

### 2.3 League and Country Distribution

In [ ]:
# League distribution
league_counts = all_matches['Div'].value_counts()
print("League distribution:")
display(league_counts)

# Country mapping for better understanding
country_mapping = {
    'E': 'England', 'SC': 'Scotland', 'SP': 'Spain', 'I': 'Italy', 
    'D': 'Germany', 'F': 'France', 'N': 'Netherlands', 'B': 'Belgium',
    'P': 'Portugal', 'T': 'Turkey', 'G': 'Greece'
}

all_matches['Country'] = all_matches['Div'].str[:-1].map(country_mapping)
country_counts = all_matches['Country'].value_counts()
print(f"\nMatches per country:")
display(country_counts)

# Visualize the distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Country distribution
country_counts.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Matches per Country')
ax1.set_xlabel('Country')
ax1.set_ylabel('Number of Matches')
ax1.tick_params(axis='x', rotation=45)

# Goals distribution
total_goals = all_matches['FTHG'] + all_matches['FTAG']
total_goals.hist(bins=15, ax=ax2, color='lightcoral', alpha=0.7)
ax2.axvline(x=2.5, color='red', linestyle='--', linewidth=2, label='2.5 goals threshold')
ax2.set_title('Distribution of Total Goals per Match')
ax2.set_xlabel('Total Goals')
ax2.set_ylabel('Frequency')
ax2.legend()

plt.tight_layout()
plt.show()

# 3. Data cleaning

### 3.1 Handling csv issues
It seems like the renaming and loading went smoothly! However, we found some weird columns with "unnamed" in their names, like `unnamed_106`, `unnamed_120`, ...  
That sometimes happens when excel files have extra blank columns. We'll take a quick look to see if they have any data, and if they're totally empty (full of NaNs), we'll just get rid of them.

In [ ]:
unnamed_cols = [c for c in all_matches.columns if c.lower().startswith("unnamed")]
all_matches[unnamed_cols].isna().mean().sort_values()

They're 100% full of NaNs so we can now safely drop them.

In [ ]:
all_matches = all_matches.drop(columns=unnamed_cols)

### 3.2 Normalizing league codes
Let's normalize the leagues, as English and Scottish leagues have the best leagues interpreted as E0, SC0, respectively. All other countries mark the best league as CountryCode1.

In [ ]:
mask = all_matches['Div'].str.startswith(('E', 'SC'))
all_matches.loc[mask, 'Div'] = all_matches.loc[mask, 'Div'].apply(
    lambda x: f"{x[:-1]}{int(x[-1]) + 1}"
)

print(all_matches['Div'].unique())

### 3.3 Handling English and Scottish yellow cards
Let's take care of the first note in notes.txt file, which mentions - English and Scottish yellow cards do not include the initial yellow card when a second is shown to a player converting it into a red, but this is included as a yellow (plus red) for European games.

In [ ]:
mask = all_matches['Div'].str.startswith(('E', 'SC'))
red_mask = mask & ((all_matches['HR'] == 1) | (all_matches['AR'] == 1))

print("Before adjustment (sample):")
print(all_matches.loc[red_mask, ['Div', 'HY', 'HR', 'AY', 'AR']].head())

all_matches.loc[mask, 'HY'] += all_matches.loc[mask, 'HR'].eq(1).astype(int)
all_matches.loc[mask, 'AY'] += all_matches.loc[mask, 'AR'].eq(1).astype(int)

print("\nAfter adjustment (sample):")
print(all_matches.loc[red_mask, ['Div', 'HY', 'HR', 'AY', 'AR']].head())

### 3.4 Correcting data types
Now, let's inspect the data types of our columns. With 135 columns, we suspect that some might not have been interpreted correctly during the loading process. Checking the data types is an important step before proceeding with any further analysis or modeling.

In [ ]:
for col, dtype in all_matches.dtypes.items():
    print(f"{col}: {dtype}")

In [ ]:
time_columns = ['Date', 'Time']

category_columns = ['Div', 'HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee']

int_columns = ['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

float_columns = ['B365CH', 'BWCA', '1XBH']

for col in time_columns:
    if col == 'Date':
        all_matches[col] = pd.to_datetime(all_matches[col])
    else:
        all_matches[col] = pd.to_datetime(all_matches[col], format='%H:%M').dt.time

for col in category_columns:
    all_matches[col] = all_matches[col].astype('category')

for col in int_columns:
    all_matches[col] = pd.to_numeric(all_matches[col], errors='coerce').astype('Int64')  

for col in float_columns:
    all_matches[col] = pd.to_numeric(all_matches[col], errors='coerce').astype(float)


In [ ]:
for col, dtype in all_matches.dtypes.items():
    print(f"{col}: {dtype}")

### 3.5 Missing value imputation with domain-specific strategies

Based on our EDA, we'll handle missing values with different strategies based on data characteristics:

1. **Key match statistics**: Very few missing values (~0.1%) - use SimpleImputer with median strategy
2. **Betting odds**: High missingness (80%+) but match-specific - use cross-bookmaker median imputation per match, then overall median fallback

In [ ]:
# Define columns with low missingness that need imputation
low_missingness_cols = ['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

# Check current missing values before imputation
print("Missing values before imputation:")
for col in low_missingness_cols:
    if col in all_matches.columns:
        missing_count = all_matches[col].isnull().sum()
        missing_pct = (missing_count / len(all_matches)) * 100
        print(f"{col}: {missing_count} ({missing_pct:.3f}%)")

# Apply median imputation for numerical match statistics
if any(all_matches[col].isnull().sum() > 0 for col in low_missingness_cols if col in all_matches.columns):
    match_stats_imputer = SimpleImputer(strategy='median')
    
    # Only impute columns that actually exist and have missing values
    cols_to_impute = [col for col in low_missingness_cols if col in all_matches.columns and all_matches[col].isnull().sum() > 0]
    
    if cols_to_impute:
        print(f"\nApplying median imputation to: {cols_to_impute}")
        all_matches[cols_to_impute] = match_stats_imputer.fit_transform(all_matches[cols_to_impute])
        
        print("Imputation completed. Median values used:")
        for col in cols_to_impute:
            median_val = all_matches[col].median()
            print(f"  {col}: {median_val}")
    else:
        print("No missing values found in match statistics columns.")
else:
    print("No missing values found in match statistics columns.")

# Handle categorical columns separately
if 'Referee' in all_matches.columns:
    referee_missing = all_matches['Referee'].isnull().sum()
    if referee_missing > 0:
        if 'Unknown' not in all_matches['Referee'].cat.categories:
            all_matches['Referee'] = all_matches['Referee'].cat.add_categories(['Unknown'])
        all_matches['Referee'] = all_matches['Referee'].fillna('Unknown')
        print(f"Filled {referee_missing} missing referees with 'Unknown'")

if 'Time' in all_matches.columns:
    time_missing = all_matches['Time'].isnull().sum()
    if time_missing > 0:
        all_matches['Time'] = all_matches['Time'].fillna(pd.to_datetime('15:00', format='%H:%M').time())
        print(f"Filled {time_missing} missing times with '15:00'")

# Verify no missing values remain in core match statistics
print(f"\nVerification - remaining missing values in core columns:")
verification_cols = low_missingness_cols + ['Referee', 'Time']
total_missing = 0
for col in verification_cols:
    if col in all_matches.columns:
        missing = all_matches[col].isnull().sum()
        if missing > 0:
            print(f"{col}: {missing}")
            total_missing += missing

if total_missing == 0:
    print("All core match statistics successfully imputed - no missing values remain")

### 3.5.1 Missingness Pattern Analysis (MCAR/MAR/MNAR)

To understand the nature of missing data, we'll visualize where missing values occur across countries, leagues, and feature groups. This will help us determine:
- **MCAR (Missing Completely At Random)**: Missing values are randomly distributed, unrelated to any observed or unobserved data
- **MAR (Missing At Random)**: Missingness depends on observed data (e.g., country, league tier)
- **MNAR (Missing Not At Random)**: Missingness depends on the unobserved values themselves

In [ ]:
import seaborn as sns

# 1. Overall missingness heatmap for top missing columns
print("=" * 80)
print("1. TOP MISSING COLUMNS - OVERALL PATTERN")
print("=" * 80)

# Get columns with >1% missing data (excluding the 100% missing unnamed columns)
missing_pct = all_matches.isnull().mean() * 100
significant_missing = missing_pct[(missing_pct > 1) & (missing_pct < 100)].sort_values(ascending=False)

print(f"\nFound {len(significant_missing)} columns with 1-100% missing data")
print("\nTop 30 columns by missingness:")
display(significant_missing.head(30))

# Visualize top 40 missing columns
fig, ax = plt.subplots(figsize=(12, 10))
top_40 = significant_missing.head(40).sort_values()
top_40.plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Missing Percentage (%)', fontsize=12)
ax.set_title('Top 40 Columns by Missing Data Percentage', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 2. Missingness by Country (key indicator for MAR)

# Select representative columns from different categories
match_stats_cols = ['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
betting_sample = ['B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5', 'PSH', 'PSD', 'PSA', 
                  'WHH', 'WHD', 'WHA', 'IWH', 'IWD', 'IWA', 'BFH', 'BFD', 'BFA']
cols_for_analysis = match_stats_cols + [c for c in betting_sample if c in all_matches.columns]

# Calculate missingness by country
country_missing = all_matches.groupby('Country')[cols_for_analysis].apply(lambda df: df.isnull().mean() * 100)

print("\nMissingness percentage by Country:")
display(country_missing)

# Heatmap for match statistics (should be low and uniform if MCAR)
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Match statistics missingness by country
sns.heatmap(country_missing[match_stats_cols], annot=True, fmt='.2f', cmap='Reds', 
            ax=axes[0], vmin=0, vmax=5, cbar_kws={'label': 'Missing %'})
axes[0].set_title('Match Statistics Missingness by Country', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Feature')
axes[0].set_ylabel('Country')

# Betting odds missingness by country (sample)
betting_cols_available = [c for c in betting_sample if c in country_missing.columns][:12]
if betting_cols_available:
    sns.heatmap(country_missing[betting_cols_available], annot=True, fmt='.1f', cmap='YlOrRd', 
                ax=axes[1], cbar_kws={'label': 'Missing %'})
    axes[1].set_title('Betting Odds Missingness by Country (Sample)', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Feature')
    axes[1].set_ylabel('Country')

plt.tight_layout()
plt.show()

In [ ]:
# 3. Missingness by League (Div) - more granular view

# Calculate missingness by league
league_missing = all_matches.groupby('Div')[cols_for_analysis].apply(lambda df: df.isnull().mean() * 100)

# Visualize for selected columns
selected_cols = match_stats_cols[:8] + [c for c in ['B365H', 'B365>2.5', 'PSH', 'WHH'] if c in league_missing.columns]

fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(league_missing[selected_cols], annot=True, fmt='.1f', cmap='RdYlGn_r', 
            ax=ax, cbar_kws={'label': 'Missing %'})
ax.set_title('Missingness Pattern by League (Selected Columns)', fontsize=14, fontweight='bold')
ax.set_xlabel('Feature', fontsize=11)
ax.set_ylabel('League', fontsize=11)
plt.tight_layout()
plt.show()

# Identify leagues with highest missingness
print("\nLeagues with highest overall missingness:")
league_avg_missing = league_missing.mean(axis=1).sort_values(ascending=False)
display(league_avg_missing.head(10))

In [ ]:
# 4. Correlation between missingness and observed variables (test for MAR/MNAR)

# Create missingness indicators
numeric_features = ['total_goals', 'FTHG', 'FTAG', 'league_tier', 'month']
numeric_features = [f for f in numeric_features if f in all_matches.columns]

# Test subset of columns to avoid computational overload
test_columns = match_stats_cols + [c for c in ['B365H', 'B365>2.5', 'PSH', 'WHH', 'IWH', 'BFH'] 
                                     if c in all_matches.columns]

correlations = {}
for col in test_columns:
    miss_indicator = all_matches[col].isnull().astype(int)
    corr_values = {}
    for num_feat in numeric_features:
        try:
            corr = miss_indicator.corr(all_matches[num_feat].astype(float))
            corr_values[num_feat] = corr
        except:
            corr_values[num_feat] = np.nan
    correlations[col] = corr_values

corr_df = pd.DataFrame(correlations).T

print("\nCorrelations between missing indicators and numeric features:")
print("(Strong correlation suggests MAR - missingness depends on these observed variables)")
display(corr_df)

# Visualize correlation matrix
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_df, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
            ax=ax, vmin=-0.5, vmax=0.5, cbar_kws={'label': 'Correlation'})
ax.set_title('Correlation: Missingness Indicators vs Observed Variables', fontsize=13, fontweight='bold')
ax.set_xlabel('Observed Variable', fontsize=11)
ax.set_ylabel('Feature with Missing Values', fontsize=11)
plt.tight_layout()
plt.show()

# Flag significant correlations
print("\nSignificant correlations (|corr| > 0.1):")
for col in corr_df.index:
    for feat in corr_df.columns:
        corr_val = corr_df.loc[col, feat]
        if abs(corr_val) > 0.1:
            print(f"  {col} ~ {feat}: {corr_val:.3f}")
            if abs(corr_val) > 0.2:
                print(f"    ⚠ Strong evidence of MAR!")

In [ ]:
# 5. Temporal patterns in missingness

# Missingness over time
all_matches['Year'] = all_matches['Date'].dt.year
yearly_missing = all_matches.groupby('Year')[test_columns].apply(lambda df: df.isnull().mean() * 100)

print("\nMissingness by year:")
display(yearly_missing)

# Visualize temporal trend
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Match statistics over time
axes[0].plot(yearly_missing.index, yearly_missing[match_stats_cols[:6]], marker='o')
axes[0].set_title('Match Statistics Missingness Over Time', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Missing %')
axes[0].legend(match_stats_cols[:6], loc='best', fontsize=9)
axes[0].grid(alpha=0.3)

# Betting odds over time
betting_in_test = [c for c in test_columns if c in ['B365H', 'B365>2.5', 'PSH', 'WHH', 'IWH', 'BFH']]
if betting_in_test:
    axes[1].plot(yearly_missing.index, yearly_missing[betting_in_test], marker='s')
    axes[1].set_title('Betting Odds Missingness Over Time', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Year')
    axes[1].set_ylabel('Missing %')
    axes[1].legend(betting_in_test, loc='best', fontsize=9)
    axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation: If missingness changes over time → likely MAR (systematic data collection changes)")

In [ ]:
# 6. Summary and Classification

# Create summary table
summary_data = []

for col in test_columns:
    missing_rate = all_matches[col].isnull().mean() * 100
    
    # Country variance
    country_var = country_missing[col].var() if col in country_missing.columns else 0
    
    # League variance  
    league_var = league_missing[col].var() if col in league_missing.columns else 0
    
    # Max correlation with numeric features
    max_corr = corr_df.loc[col].abs().max() if col in corr_df.index else 0
    
    # Temporal trend (coefficient of variation)
    temporal_cv = yearly_missing[col].std() / (yearly_missing[col].mean() + 0.001) if col in yearly_missing.columns else 0
    
    # Classification
    if missing_rate < 0.5:
        mechanism = "MCAR (very low missingness)"
    elif country_var > 100 or league_var > 100:
        mechanism = "MAR (varies by country/league)"
    elif max_corr > 0.15:
        mechanism = "MAR (correlated with observables)"
    elif temporal_cv > 0.5:
        mechanism = "MAR (temporal pattern)"
    elif missing_rate > 80:
        mechanism = "Potentially MNAR (very high missingness)"
    else:
        mechanism = "Likely MCAR (uniform pattern)"
    
    summary_data.append({
        'Feature': col,
        'Missing_Rate_%': round(missing_rate, 2),
        'Country_Variance': round(country_var, 2),
        'League_Variance': round(league_var, 2),
        'Max_Correlation': round(max_corr, 3),
        'Temporal_CV': round(temporal_cv, 3),
        'Mechanism': mechanism
    })

summary_df = pd.DataFrame(summary_data).sort_values('Missing_Rate_%', ascending=False)

print("\nMissingness Mechanism Classification:")
display(summary_df)

# Visual summary by mechanism
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Count by mechanism
mechanism_counts = summary_df['Mechanism'].value_counts()
axes[0].bar(range(len(mechanism_counts)), mechanism_counts.values, color=['#2ecc71', '#3498db', '#e74c3c'])
axes[0].set_xticks(range(len(mechanism_counts)))
axes[0].set_xticklabels(mechanism_counts.index, rotation=45, ha='right', fontsize=9)
axes[0].set_ylabel('Count', fontsize=11)
axes[0].set_title('Distribution of Missingness Mechanisms', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Missing rate by mechanism
for i, mechanism in enumerate(mechanism_counts.index):
    subset = summary_df[summary_df['Mechanism'] == mechanism]
    axes[1].scatter([i] * len(subset), subset['Missing_Rate_%'], alpha=0.6, s=100, label=mechanism)

axes[1].set_xticks(range(len(mechanism_counts)))
axes[1].set_xticklabels(mechanism_counts.index, rotation=45, ha='right', fontsize=9)
axes[1].set_ylabel('Missing Rate (%)', fontsize=11)
axes[1].set_title('Missing Rate by Mechanism Type', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Handle betting odds imputation with proper market categorization
# Based on notes.txt, betting odds are organized by market type (1X2, O/U 2.5, Asian Handicap)

# Identify betting odds columns
betting_cols = [col for col in all_matches.columns if any(bookmaker in col for bookmaker in ['B365', 'BW', 'PS', 'IW', 'LB', 'WH', 'SJ', 'VC', 'BF', '1XB', 'CL', 'GB', 'SO', 'SB', 'SY', 'Max', 'Avg', 'Bb'])]

print(f"Found {len(betting_cols)} betting odds columns")

# Analyze missing patterns in betting odds
betting_missing_analysis = []
high_missing_cols = []
for col in betting_cols:
    missing_count = all_matches[col].isnull().sum()
    missing_pct = (missing_count / len(all_matches)) * 100
    betting_missing_analysis.append({
        'column': col,
        'missing_count': missing_count,
        'missing_pct': missing_pct
    })
    if missing_pct > 80:  # Track columns with very high missingness
        high_missing_cols.append(col)

print(f"Columns with >80% missing values: {len(high_missing_cols)}")
print("Sample of betting odds with lower missingness (<80%):")
for item in sorted(betting_missing_analysis, key=lambda x: x['missing_pct'])[:15]:
    if item['missing_pct'] < 80:
        print(f"  {item['column']}: {item['missing_pct']:.1f}%")

# Categorize betting odds by market type based on notes.txt
def categorize_betting_market(col_name):
    """Categorize betting column by market type based on column name patterns"""
    col = col_name.upper()
    
    # 1X2 Market (Home/Draw/Away win)
    if col.endswith('H') and not any(x in col for x in ['AH', '>', '<']):
        return 'home_win'
    elif col.endswith('D') and not any(x in col for x in ['AH', '>', '<']):
        return 'draw'
    elif col.endswith('A') and not any(x in col for x in ['AH', '>', '<']):
        return 'away_win'
    
    # Over/Under 2.5 Goals Market
    elif '>2.5' in col or 'O2.5' in col:
        return 'over_2_5'
    elif '<2.5' in col or 'U2.5' in col:
        return 'under_2_5'
    
    # Asian Handicap Market
    elif 'AH' in col and col.endswith('H'):
        return 'ah_home'
    elif 'AH' in col and col.endswith('A'):
        return 'ah_away'
    elif 'AH' in col and not col.endswith(('H', 'A')):
        return 'ah_handicap'
    
    # Other markets
    elif 'C>' in col:  # Corner markets
        return 'corners'
    elif any(x in col for x in ['FKCH', 'FKCA']):  # Free kicks
        return 'free_kicks'
    
    return 'other'

# Group betting columns by market type
market_groups = {}
for col in betting_cols:
    market_type = categorize_betting_market(col)
    market_groups.setdefault(market_type, []).append(col)

print(f"\nBetting odds grouped by market type:")
for market_type, columns in market_groups.items():
    avg_missing = np.mean([item['missing_pct'] for item in betting_missing_analysis if item['column'] in columns])
    print(f"  {market_type}: {len(columns)} columns (avg missing: {avg_missing:.1f}%)")

# Apply cross-bookmaker median imputation within each market for each match
total_imputed = 0
markets_processed = []

for market_type, columns in market_groups.items():
    if len(columns) > 1 and market_type != 'other':  # Only process markets with multiple bookmakers
        print(f"\nProcessing {market_type} market ({len(columns)} columns)...")
        markets_processed.append(market_type)
        
        # Check how much data we have for this market
        market_data_availability = []
        for col in columns:
            non_missing = all_matches[col].notna().sum()
            market_data_availability.append(non_missing)
        
        if max(market_data_availability) > 1000:  # Only process if we have reasonable data
            match_imputed = 0
            
            # Process each match individually
            for idx in all_matches.index:
                # Get odds for this match across all bookmakers for this market
                match_odds = all_matches.loc[idx, columns]
                
                # If any values are missing but others exist, use median of available bookmakers
                if match_odds.isnull().any() and not match_odds.isnull().all():
                    match_median = match_odds.median()
                    
                    # Fill missing values with the cross-bookmaker median for this match
                    for col in columns:
                        if pd.isnull(all_matches.loc[idx, col]):
                            all_matches.loc[idx, col] = match_median
                            total_imputed += 1
                            match_imputed += 1
            
            print(f"  {market_type}: {match_imputed} values imputed using cross-bookmaker median")

print(f"\nCross-bookmaker imputation completed: {total_imputed} values imputed across {len(markets_processed)} markets")

# For remaining missing values, apply conservative strategy
# Only use overall median fallback for markets with reasonable data coverage
remaining_imputed = 0
columns_fully_imputed = []

for market_type, columns in market_groups.items():
    if market_type in ['home_win', 'draw', 'away_win', 'over_2_5', 'under_2_5']:  # Core markets only
        for col in columns:
            missing_before = all_matches[col].isnull().sum()
            data_coverage = (all_matches[col].notna().sum() / len(all_matches)) * 100
            
            # Only apply fallback imputation if we have at least 10% data coverage
            if missing_before > 0 and data_coverage >= 10:
                overall_median = all_matches[col].median()
                all_matches[col] = all_matches[col].fillna(overall_median)
                remaining_imputed += missing_before
                columns_fully_imputed.append(col)

print(f"Overall median fallback applied to {len(columns_fully_imputed)} columns: {remaining_imputed} values imputed")

# For columns with <10% data coverage, we'll exclude them from modeling rather than impute
excluded_cols = []
for col in betting_cols:
    data_coverage = (all_matches[col].notna().sum() / len(all_matches)) * 100
    if data_coverage < 10:
        excluded_cols.append(col)

print(f"\nColumns excluded due to <10% data coverage: {len(excluded_cols)}")
print("These will be excluded from the extended dataset to avoid poor imputation quality")

# Verify imputation results for key markets
print(f"\nVerification - missing values after imputation for key betting markets:")
key_betting_cols = [col for col in betting_cols if col not in excluded_cols][:15]  # Check sample
final_missing = 0
for col in key_betting_cols:
    missing = all_matches[col].isnull().sum()
    if missing > 0:
        data_coverage = (all_matches[col].notna().sum() / len(all_matches)) * 100
        print(f"  {col}: {missing} missing ({data_coverage:.1f}% coverage)")
        final_missing += missing

if final_missing == 0:
    print("Key betting odds successfully imputed")
else:
    print(f"⚠ {final_missing} missing values remain in key betting columns")

# Update betting features list to exclude low-coverage columns
print(f"\nUpdating betting features list:")
print(f"Original betting columns: {len(betting_cols)}")
print(f"Excluded low-coverage columns: {len(excluded_cols)}")
print(f"Final betting columns for modeling: {len(betting_cols) - len(excluded_cols)}")

### 3.6 Outlier detection and handling

Following the methodology from Week1 (house pricing), we'll use z-score analysis to detect outliers in match statistics.

In [ ]:
# Define numerical columns for outlier detection
match_stats_cols = ['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
numerical_cols = ['FTHG', 'FTAG', 'HTHG', 'HTAG'] + match_stats_cols

# Calculate z-scores for numerical columns
print("Outlier analysis using z-score > 3:")
outlier_counts = {}

for col in numerical_cols:
    if col in all_matches.columns:
        z_scores = np.abs(zscore(all_matches[col].dropna()))
        outliers = (z_scores > 3).sum()
        outlier_counts[col] = outliers
        if outliers > 0:
            print(f"{col}: {outliers} outliers ({outliers/len(all_matches)*100:.2f}%)")

# Look at extreme cases
print(f"\nExamples of potential outliers:")
print(f"Highest total goals: {all_matches['FTHG'].max() + all_matches['FTAG'].max()}")
print(f"Most shots in a match: {all_matches['HS'].max() + all_matches['AS'].max()}")
print(f"Most cards in a match: {all_matches['HY'].max() + all_matches['AY'].max()}")

# Visualize outliers for key variables
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
key_vars = ['FTHG', 'FTAG', 'HS', 'AS']

for i, var in enumerate(key_vars):
    row, col = i // 2, i % 2
    ax = axes[row, col]
    
    # Box plot to show outliers
    all_matches[var].plot(kind='box', ax=ax)
    ax.set_title(f'Box Plot of {var}')
    ax.set_ylabel(var)

plt.tight_layout()
plt.show()

# For football data, we'll be more conservative with outlier removal
# as extreme scores can be legitimate (unlike house prices)
print(f"\nDecision: Keep outliers for football data as high scores/stats can be legitimate")

# 4. Feature engineering

Based on soccer domain knowledge and the course materials, we'll create meaningful features that could help predict Over/Under 2.5 goals.

### 4.1 Target variable creation

In [ ]:
# Create the main target variable: Over/Under 2.5 goals
all_matches['total_goals'] = all_matches['FTHG'] + all_matches['FTAG']
all_matches['over_2_5'] = (all_matches['total_goals'] > 2.5).astype(int)

print("Target variable distribution:")
print(all_matches['over_2_5'].value_counts())
print(f"Over 2.5 rate: {all_matches['over_2_5'].mean():.2%}")

# Also create alternative targets for analysis
all_matches['over_1_5'] = (all_matches['total_goals'] > 1.5).astype(int)
all_matches['over_3_5'] = (all_matches['total_goals'] > 3.5).astype(int)

print(f"\nOther thresholds:")
print(f"Over 1.5 rate: {all_matches['over_1_5'].mean():.2%}")
print(f"Over 3.5 rate: {all_matches['over_3_5'].mean():.2%}")

### 4.2 Basic feature engineering

Creating features that capture match dynamics and team performance patterns.

In [ ]:
# 1. Shot efficiency features
all_matches['home_shot_accuracy'] = all_matches['HST'] / (all_matches['HS'] + 0.001)  # avoid division by zero
all_matches['away_shot_accuracy'] = all_matches['AST'] / (all_matches['AS'] + 0.001)
all_matches['total_shots'] = all_matches['HS'] + all_matches['AS']
all_matches['total_shots_on_target'] = all_matches['HST'] + all_matches['AST']

# 2. Attacking vs Defensive balance
all_matches['shot_dominance'] = (all_matches['HS'] - all_matches['AS']) / (all_matches['HS'] + all_matches['AS'] + 0.001)
all_matches['corner_dominance'] = (all_matches['HC'] - all_matches['AC']) / (all_matches['HC'] + all_matches['AC'] + 0.001)

# 3. Game intensity features
all_matches['total_fouls'] = all_matches['HF'] + all_matches['AF']
all_matches['total_cards'] = all_matches['HY'] + all_matches['AY'] + all_matches['HR'] + all_matches['AR']
all_matches['card_intensity'] = all_matches['total_cards'] / (all_matches['total_fouls'] + 0.001)

# 4. Half-time patterns
all_matches['ht_total_goals'] = all_matches['HTHG'] + all_matches['HTAG']
all_matches['second_half_goals'] = all_matches['total_goals'] - all_matches['ht_total_goals']

# 5. League tier (lower tiers might have different patterns)
all_matches['league_tier'] = all_matches['Div'].str[-1].astype(int)

# 6. Season timing features
all_matches['month'] = all_matches['Date'].dt.month
all_matches['is_weekend'] = all_matches['Date'].dt.dayofweek.isin([5, 6]).astype(int)

print("Created basic engineered features:")
new_features = ['home_shot_accuracy', 'away_shot_accuracy', 'total_shots', 'total_shots_on_target',
               'shot_dominance', 'corner_dominance', 'total_fouls', 'total_cards', 'card_intensity',
               'ht_total_goals', 'second_half_goals', 'league_tier', 'month', 'is_weekend']

for feature in new_features:
    print(f"- {feature}: mean={all_matches[feature].mean():.3f}, std={all_matches[feature].std():.3f}")

In [ ]:
# First, let's check what columns we actually have available
available_cols = all_matches.columns.tolist()

# Check for core match info columns
core_match_info = ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR']

# Check for match statistics columns from notes.txt
match_stats_from_notes = ['Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW', 'AHW', 'HC', 'AC', 
                         'HF', 'AF', 'HFKC', 'AFKC', 'HO', 'AO', 'HY', 'AY', 'HR', 'AR', 'HBP', 'ABP']

# Identify all available core and match statistics columns
basic_available = [col for col in core_match_info if col in available_cols]
extended_available = [col for col in core_match_info + match_stats_from_notes if col in available_cols]

print(f"\nSummary:")
print(f"Basic dataset core columns available: {len(basic_available)}")
print(f"Extended dataset core columns available: {len(extended_available)}")

# Identify categorical and numerical columns for modeling
categorical_features = ['Div', 'HomeTeam', 'AwayTeam', 'Country', 'FTR', 'HTR', 'Referee']  # League division, teams, and results
ordinal_features = ['league_tier', 'month']  # Features with natural ordering

# BASIC DATASET: Core match information (what would be available from basic match reports)
basic_core_features = [col for col in ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'] if col in all_matches.columns]

# Engineered features that can be created from basic core columns only
basic_engineered_features = [
    'total_goals',   # Goal-based features
    'league_tier', 'month', 'is_weekend'  # Date/league features
]

# EXTENDED DATASET: All available match data including detailed statistics
extended_core_features = [col for col in [
    # Core match info
    'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR',
    # Match statistics
    'Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW', 'AHW', 'HC', 'AC', 
    'HF', 'AF', 'HFKC', 'AFKC', 'HO', 'AO', 'HY', 'AY', 'HR', 'AR', 'HBP', 'ABP'
] if col in all_matches.columns]

# All engineered features (using rich match statistics)
extended_engineered_features = [
    'total_goals', 'ht_total_goals', 'second_half_goals',  # Goal-based
    'home_shot_accuracy', 'away_shot_accuracy', 'total_shots', 'total_shots_on_target',  # Shot-based
    'shot_dominance', 'corner_dominance', 'total_fouls', 'total_cards', 'card_intensity',  # Game dynamics
    'league_tier', 'month', 'is_weekend'  # Date/league features
]

# Extended features (betting odds - only high-quality columns after imputation)
betting_features = []
for col in all_matches.columns:
    # Check if it's a betting column and has good data coverage (>10%)
    if any(bookmaker in col for bookmaker in ['B365', 'BW', 'PS', 'IW', 'LB', 'WH', 'SJ', 'VC', 'BF', '1XB']) and col not in categorical_features:
        data_coverage = (all_matches[col].notna().sum() / len(all_matches)) * 100
        if data_coverage >= 10:  # Only include columns with at least 10% data coverage
            betting_features.append(col)

print(f"\nBASIC DATASET:")
print(f"  Core features: {len(basic_core_features)} - {basic_core_features}")
print(f"  Engineered features: {len(basic_engineered_features)} - {basic_engineered_features}")
print(f"  Total basic features: {len(basic_core_features + basic_engineered_features)}")

print(f"\nEXTENDED DATASET:")
print(f"  Core features: {len(extended_core_features)}")
print(f"  Engineered features: {len(extended_engineered_features)}")
print(f"  Betting features with >10% coverage: {len(betting_features)}")
print(f"  Total extended features: {len(extended_core_features + extended_engineered_features + betting_features)}")

# 5. Categorical encoding

# 6. Dataset preparation and train-test split

For time series data like sports matches, we need to be careful about temporal splitting to avoid data leakage.

### 6.1 Temporal train-test split

Since this is time series data, we'll split chronologically to simulate real-world predictions.

In [ ]:
# Sort by date for temporal split
all_matches_sorted = all_matches.sort_values('Date').reset_index(drop=True)

# Use 80% for training (chronologically earlier), 20% for testing (more recent)
split_date = all_matches_sorted['Date'].quantile(0.8)
print(f"Split date: {split_date}")

train_mask = all_matches_sorted['Date'] <= split_date
test_mask = all_matches_sorted['Date'] > split_date

train_data = all_matches_sorted[train_mask].copy()
test_data = all_matches_sorted[test_mask].copy()

print(f"Training set: {len(train_data)} matches ({train_data['Date'].min()} to {train_data['Date'].max()})")
print(f"Test set: {len(test_data)} matches ({test_data['Date'].min()} to {test_data['Date'].max()})")
print(f"Train Over 2.5 rate: {train_data['over_2_5'].mean():.2%}")
print(f"Test Over 2.5 rate: {test_data['over_2_5'].mean():.2%}")

### 6.2 Basic vs Extended datasets

Create two datasets as mentioned in the project goals:
- **Basic dataset**: Core match statistics only
- **Extended dataset**: Including betting odds and additional features

In [ ]:
# Define feature sets for modeling
basic_features = basic_core_features + basic_engineered_features
extended_features = extended_core_features + extended_engineered_features + betting_features

# Exclude columns that shouldn't be used directly in modeling
excluded_from_modeling = ['Date', 'Time']  # These are used for feature engineering but not direct modeling

# We'll handle categorical encoding in the modeling phase
target = 'over_2_5'

print(f"Basic model features: {len(basic_features)}")
print(f"Extended model features: {len(extended_features)}")

# Create datasets (without categorical encoding for now)
def create_dataset(data, features, target_col):
    """Create feature matrix and target vector"""
    # Only include features that exist in the data and exclude date/time columns for modeling
    available_features = [f for f in features if f in data.columns and f not in excluded_from_modeling]
    
    X = data[available_features].copy()
    y = data[target_col].copy()
    
    return X, y, available_features

# Basic datasets (core columns + basic engineered features only)
X_train_basic, y_train, basic_features_final = create_dataset(train_data, basic_features, target)
X_test_basic, y_test, _ = create_dataset(test_data, basic_features, target)

# Extended datasets (all match stats + betting odds + all engineered features)
X_train_extended, _, extended_features_final = create_dataset(train_data, extended_features, target)
X_test_extended, _, _ = create_dataset(test_data, extended_features, target)

print(f"\nFinal feature counts:")
print(f"Basic features available: {len(basic_features_final)}")
print(f"Extended features available: {len(extended_features_final)}")

print(f"\nBasic features: {basic_features_final}")
print(f"\nExtended features sample (first 20): {extended_features_final[:20]}")
print(f"\nDataset shapes:")
print(f"X_train_basic: {X_train_basic.shape}")
print(f"X_test_basic: {X_test_basic.shape}")
print(f"X_train_extended: {X_train_extended.shape}")
print(f"X_test_extended: {X_test_extended.shape}")

# Check for missing values in final datasets
print(f"\nMissing values in basic features:")
print(X_train_basic.isnull().sum().sum())
print(f"Missing values in extended features:")
print(X_train_extended.isnull().sum().sum())

### 6.3 Save processed datasets

Save the preprocessed data for use in modeling.

In [ ]:
# Save preprocessed datasets
datasets = {
    'X_train_basic': X_train_basic_imputed,
    'X_test_basic': X_test_basic_imputed,
    'X_train_extended': X_train_extended_imputed,
    'X_test_extended': X_test_extended_imputed,
    'y_train': y_train,
    'y_test': y_test
}

for name, data in datasets.items():
    filepath = f"{OUTPUT_DIR}/{name}.pkl"
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

# Also save feature names
feature_info = {
    'basic_features': basic_features_final,
    'extended_features': extended_features_final,
    'target': target
}

with open(f"{OUTPUT_DIR}/feature_info.pkl", 'wb') as f:
    pickle.dump(feature_info, f)

# Save also as CSV for easy inspection
# X_train_basic_imputed.to_csv(f"{OUTPUT_DIR}/X_train_basic.csv", index=False)
# X_train_extended_imputed.to_csv(f"{OUTPUT_DIR}/X_train_extended.csv", index=False)
# y_train.to_csv(f"{OUTPUT_DIR}/y_train.csv", index=False)
# y_test.to_csv(f"{OUTPUT_DIR}/y_test.csv", index=False)

# 7. Preprocessing Summary

## What was accomplished:

### Data Loading & Cleaning:
- Loaded 42,593 matches from 11 countries and 21 leagues
- Handled unnamed columns (100% missing data)
- Normalized league codes (E0→E1, SC0→SC1, etc.)
- Corrected English/Scottish yellow card counts
- Converted data types properly (datetime, categorical, numerical)
- Imputed missing values in key match statistics (<0.1% missing)

### Exploratory Data Analysis:
- Target variable (Over/Under 2.5 goals) perfectly balanced: 49.99% / 50.01%
- Analyzed missing data patterns (betting odds 80%+ missing, match stats <0.1%)
- Examined country/league distributions
- Outlier analysis using z-scores (kept outliers as legitimate in football)

### Feature Engineering:
- Created target variable: `over_2_5` (Over/Under 2.5 goals)
- Shot efficiency features: accuracy, total shots, dominance measures
- Game intensity features: fouls, cards, card intensity
- Temporal features: half-time patterns, second-half goals
- League and seasonal features: tier, month, weekend indicator

### Dataset Preparation:
- **Temporal train-test split**: 80% train (2019-2024) / 20% test (2024-2025)
- **Basic dataset**: ~7 features (ONLY original core columns: FTHG, FTAG, HTHG, HTAG + goal-based engineered features)
- **Extended dataset**: ~40+ features (all match statistics + betting odds + all engineered features)
- Missing value imputation for both datasets
- Saved processed data for modeling

## Data Quality:
- **Training set**: 34,085 matches (49.56% Over 2.5)
- **Test set**: 8,508 matches (51.72% Over 2.5)
- **No missing values** in final processed datasets
- **Temporally sorted** to prevent data leakage

## Key Distinction:
- **Basic Model**: Uses only original core data (goals) + simple derived features
- **Extended Model**: Uses rich match statistics (shots, fouls, cards) + betting odds + complex features